## Test database connection locally
Must have data ingested. Use ./first_db_connection.ipynb if initializing database

In [1]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
import os
import json
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey

load_dotenv('../../shared/.env')

# database env
URL = os.getenv('COSMOSDB_ENDPOINT')  
MASTER_KEY = os.getenv('COSMOSDB_KEY')

# init client
client = cosmos_client.CosmosClient(URL, {'masterKey': MASTER_KEY}, user_agent="", user_agent_overwrite=True)

# Create a database (if it doesn't exist)
database_name = 'MojovaDB'
database = client.get_database_client(database_name)

# Create a container (if it doesn't exist)
container_name = 'Analytics'
try:
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path='/medallion'),
        offer_throughput=400
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

import pandas as pd
query = "SELECT * FROM c WHERE c['/medallion'] = 'bronze_nes'"
data = container.query_items(query=query, enable_cross_partition_query=True)

# data contains iterator object
data = [item for item in data]

df = pd.DataFrame(data)
df

,org_id,unnamed:_1,tyoyksikko,tyonkuva,unnamed:_4,tyosuhde,koulutus,tyovuoro,tyosuhteen_pituus,uskon_org_paamaariin,...,org_innostaa,tyoskentelen_3v_todnak,valmis_panostamaan,id,/medallion,_rid,_self,_etag,_attachments,_ts
0,117,Leikkaus-anestesia,PERIOPAAM,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,3.0,6,5.0,...,5,4,5,a03e5d8b-18f8-4ca7-bbe0-7bf1404bc892,bronze_nes,1kAqANsuEkozAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005421-0000-4700-0000-6730a7c30000""",attachments/,1731241923
1,117,Päivystyspkl,EALAPSAIK,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,2.0,4,4.0,...,2,6,4,96c682ee-feed-4ab5-a189-b6cd547e7fee,bronze_nes,1kAqANsuEko0AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005521-0000-4700-0000-6730a7c30000""",attachments/,1731241923
2,117,Ensihoito,ENSIHOITO,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,3.0,5,5.0,...,3,5,5,f05d352f-857b-4367-aa74-afba58d78e79,bronze_nes,1kAqANsuEko1AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005621-0000-4700-0000-6730a7c30000""",attachments/,1731241923
3,117,Päivystyspkl,EALAPSAIK,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,2.0,3,4.0,...,5,3,5,82885b13-b530-4269-9a17-0f32f44d285b,bronze_nes,1kAqANsuEko2AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005721-0000-4700-0000-6730a7c30000""",attachments/,1731241923
4,117,Osasto B,OSHOITAJA,25,"Osastonhoitaja, tulosyksikköpäällikkö, palvelu...",1.0,2.0,1.0,6,4.0,...,5,5,5,0f740562-68e3-4514-9222-94ffed7d93a5,bronze_nes,1kAqANsuEko3AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f005821-0000-4700-0000-6730a7c30000""",attachments/,1731241923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,117,Leikkaus-anestesia,PERIOPAAM,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,3.0,6,4.0,...,4,6,6,30f79b8b-1dd4-48f3-99ef-58965e23fb90,bronze_nes,1kAqANsuEkrxAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002c22-0000-4700-0000-6730a7cf0000""",attachments/,1731241935
191,117,Osasto B,AIKKIRMUU,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,3.0,5,6.0,...,6,6,6,65a4ea40-defe-4d5b-a39e-b707ff126736,bronze_nes,1kAqANsuEkryAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002d22-0000-4700-0000-6730a7cf0000""",attachments/,1731241935
192,117,Osasto A,AIKMEDMUU,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,3.0,3.0,5,6.0,...,6,1,6,811840ef-26e9-40b2-bb4a-ccf5fde69e7d,bronze_nes,1kAqANsuEkrzAAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002e22-0000-4700-0000-6730a7cf0000""",attachments/,1731241935
193,117,Teho- ja valvontaosasto,AIKTEHOHO,16,"Sairaanhoitaja, audionomi, terveydenhoitaja",1.0,4.0,3.0,6,4.0,...,6,6,6,ffc81623-b374-4f64-b658-0445c9a53c97,bronze_nes,1kAqANsuEkr0AAAAAAAAAA==,dbs/1kAqAA==/colls/1kAqANsuEko=/docs/1kAqANsuE...,"""6f002f22-0000-4700-0000-6730a7cf0000""",attachments/,1731241935
